## Preliminary data analysis metrics

> An explanation of the central idea behind your final project (What is the idea? Why is it interesting? Which datasets did you need to explore the idea? How did you download them?)

Required imports

In [33]:
# clean up outputs from warnings
import warnings
warnings.filterwarnings("ignore")

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scraper
import utils
import numpy as np
import pandas as pd

from nltk import FreqDist
import pathlib
from multiprocessing import Pool
import pickle

plt.rcParams['figure.figsize'] = [10, 8]

If something related to tqdm fails, run:
> `pip install ipywidgets widgetsnbextension pandas-profiling`

Changes?
> `pip freeze > requirements.txt`

## Downloading the data

In [3]:
category_titles = [
    "Category:Wikipedia_level-1_vital_articles",
    "Category:Wikipedia_level-2_vital_articles",
    "Category:Wikipedia_level-3_vital_articles",
    #"Category:Wikipedia_level-4_vital_articles",
    #"Category:Wikipedia_level-5_vital_articles"
]

page_graph, infos  = await scraper.scrape_wiki(category_titles)

Creating graph: 100%|██████████| 7358/7358 [00:00<00:00, 10015.45it/s]


Total edges: 210653


A walk-through of your preliminary data analysis, addressing:
> - What is the total size of your data? (MB, number of pages, other variables, etc.)

In [4]:
infos.keys()

dict_keys(['titles', 'archive_titles'])

In [5]:
talk_pages = [title for title in page_graph.nodes if title[:5] == "Talk:"]
users = [title for title in page_graph.nodes if title[:5] == "User:"]
print("Number of pages in the vital articles dataset:", len(infos["titles"]))
print("Number of related archived pages:", len(infos["archive_titles"]))
print("Number of users found in relation to the dataset:", len(users))
print("No. nodes", page_graph.number_of_nodes())
print("No. links:", page_graph.number_of_edges())

Number of pages in the vital articles dataset: 1001
Number of related archived pages: 6357
Number of users found in relation to the dataset: 79523
No. nodes 81525
No. links: 162129


> - What is the network you will be analyzing? (number of nodes? number of links?, degree distributions, what are node attributes?, etc.)

In [ ]:
plt.rcParams['font.sans-serif'] = 'DejaVu Sans'

graph = page_graph.copy()

# Only keep users with a high degree
for node in page_graph.nodes(data=True):
    if "page_class" in node[1]:
        if node[1]["page_class"] == "user":
            if page_graph.out_degree(node[0]) <= 10:
                graph.remove_node(node[0])
    else:
        graph.remove_node(node[0])

# Remove outliers
cc = nx.weakly_connected_components(graph)
largest_c = max(cc, key=lambda x: len(x))
rsubgraph = nx.subgraph(graph, largest_c)

# Positions (currently unused)
#pos = nx.nx_agraph.graphviz_layout(rsubgraph, prog="neato")

# Color and size according to coast and degree
color_talk = "#0000FF"
color_user = "#FF0000"
node_colors = [color_talk if node[1]["page_class"] == "talk" else color_user for node in rsubgraph.nodes(data=True)]
node_sizes = [rsubgraph.degree(node) for node in rsubgraph.nodes]

nx.draw(rsubgraph, with_labels=True, font_weight='light', font_size=5, node_size=node_sizes, width=.1, edge_color="#555555", arrowsize=2, node_color=node_colors)

In [ ]:
# sorted(rsubgraph.degree, key=lambda item: item[1], reverse=True)[:10]
import pandas as pd

degrees = pd.DataFrame(rsubgraph.degree, columns=["Node", "Degree"])
degrees.head()

In [ ]:
degrees["PageType"] = ["User" if node[:5] == "User:" else "Talk" for node in degrees.Node ]
users = degrees[degrees.PageType == "User"]
potential_bots = users[users["Node"].str.contains('bot', case=False)]
print(f"{len(potential_bots)} users found with with bot in their name:")
print(",\n".join(potential_bots.Node.values))

After evaluation we found that `User:Botteville`, `User:KP Botany` and `User:NinjaRobotPirate` are human users. Therefore we can filter out the bots by name.


In [ ]:
bot_names = ["User:Community Tech bot", "User:PrimeBOT", "User:InternetArchiveBot", "User:AnomieBOT", "User:RMCD bot", "User:Cyberbot II", "User:CommonsNotificationBot",
"User:LinkBot", "User:FairuseBot", "User:BetacommandBot", "User:Legobot", "User:DumZiBoT"]

human_users = users[~users["Node"].isin(bot_names)]
n = 10
top_n_human_df = human_users.sort_values(["Degree"], ascending=False).head(n)
top_n_human_df

Make subgraph with users and their related pages.



In [ ]:
top_n_human_names = top_n_human_df.Node.values
top_users_graph = graph.subgraph(sum([list(graph.neighbors(node)) + [node] for node in top_n_human_names], []))
pages = [node for node in top_users_graph.nodes if node not in top_n_human_names] 

node_colors = [color_talk if node[1]["page_class"] == "talk" else color_user for node in top_users_graph.nodes(data=True)]
node_sizes = [rsubgraph.degree(node) for node in top_users_graph.nodes]

In [ ]:
degrees = dict(top_users_graph.degree())
sorted_nodes = sorted(degrees, key=degrees.get, reverse=True)
top_nodes_count = 10
top_nodes = [node for node in sorted(degrees, key=degrees.get, reverse=True) if node[:5] == "Talk:"][:top_nodes_count]
labels = {node: node for node in top_nodes}

In [ ]:
from fa2 import ForceAtlas2

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=2.0,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=3.0,
                        strongGravityMode=False,
                        gravity=0.1,

                        # Log
                        verbose=True)
pos=forceatlas2.forceatlas2_networkx_layout(top_users_graph, pos=None, iterations=2000)
    
nx.draw(top_users_graph, pos=pos, node_color=node_colors, node_size=node_sizes, edge_color="#999999", width=0.3, alpha=0.5)
plt.show()

In [ ]:
pos = nx.spring_layout(top_users_graph)
label_pos = {node: (pos[node][0], pos[node][1] + 0.15) for node in top_nodes}
nx.draw(top_users_graph, pos=pos, node_color=node_colors, node_size=node_sizes, edge_color="#999999", width=0.3, alpha=0.5, with_labels=False)
nx.draw_networkx_labels(top_users_graph, pos, labels=labels, font_size=10, font_color='black')
plt.show()

## Basic Analysis

In [ ]:
import pandas as p
import matplotlib.pyplot as plt
import powerlaw
import scipy.stats as sps

# Basic Statistics

count_nodes = len(page_graph)
count_edges = len(page_graph.edges())

# Create degree statistic dicts
degrees = dict(page_graph.degree())
in_degrees = dict(page_graph.in_degree())
out_degrees = dict(page_graph.out_degree())

talk_page_in_degrees = {k: v for k, v in in_degrees.items() if page_graph.nodes[k]["page_class"] == "talk"}
user_out_degrees = {k: v for k, v in out_degrees.items() if page_graph.nodes[k]["page_class"] == "user"}

def find_top(n, stat_dict):
    degrees_pages = []
    degrees_users = []
    top_overall = ""

    for page, degree in dict(sorted(stat_dict.items(), key=lambda item: item[1], reverse=True)).items():
        if top_overall == "":
            top_overall = page + " - " + str(degree)

        # stat dicts don't distinguish between east/west, so we'll do that here
        if len(degrees_pages) < n and page_graph.nodes[page]["page_class"] == "talk": 
            degrees_pages.append(page + " - " + str(degree))
        elif len(degrees_users) < n and page_graph.nodes[page]["page_class"] == "user":
            degrees_users.append(page + " - " + str(degree))

        if len(degrees_pages) >= n and len(degrees_users) >= n:
            break  # found all top v
    
    return degrees_pages, degrees_users, top_overall

degrees_pages, degrees_users, top_overall = find_top(10, degrees)

print("Number of nodes: " + str(count_nodes))
print("Number of links: " + str(count_edges))

print()
print("Highest degrees for pages:")
print("> Overall:")
print(top_overall)
print("> Pages:")
print("\n".join(degrees_pages))
print("> Users:")
print("\n".join(degrees_users))


# Degree multiplicities
in_degrees_counts = p.Series(talk_page_in_degrees.values()).value_counts()
out_degrees_counts = p.Series(user_out_degrees.values()).value_counts()

max_degree = max([max(in_degrees_counts.index), max(out_degrees_counts.index)])
max_multiplicity = max([max(in_degrees_counts.values), max(out_degrees_counts.values)])
range_x = range(1, max_degree + 1)

in_degrees_counts_interp = in_degrees_counts.reindex(range(max_degree+1), fill_value=0).sort_index()
out_degrees_counts_interp = out_degrees_counts.reindex(range(max_degree+1), fill_value=0).sort_index()

# Exponents
fit_in = powerlaw.Fit(in_degrees_counts.sort_index().values, verbose=False)
fit_out = powerlaw.Fit(out_degrees_counts.sort_index().values, verbose=False)

exp_in = fit_in.alpha
exp_out = fit_out.alpha

print("Exponents:")
print("In-degrees: " + str(exp_in) + " sigma: " + str(fit_in.sigma))
print("Out-degrees: " + str(exp_out) + " sigma: " + str(fit_out.sigma))

In [ ]:
import math
def fpl(x, a):
    return x ** (-a)

# Plots
fig, axs = plt.subplots(2, 2)

# Sturges rule
no_bins_sturges = int(1 + math.log(len(talk_page_in_degrees.values()), 2))

axs[0, 0].scatter(in_degrees_counts.index, in_degrees_counts.values, s=5, label='Data')
#hist, bin_edges, _ = axs[0, 0].hist(talk_page_in_degrees.values(), bins=no_bins_sturges, edgecolor='white', label='Data')
#axs[0, 0].set_xticks(bin_edges)
#axs[0, 0].set_xticklabels(['%.0f' % val for val in bin_edges], rotation=45)
axs[0,0].set_title('Multiplicity of In-degrees for Talk pages')
axs[0,0].legend()

axs[0,1].scatter(in_degrees_counts.index, in_degrees_counts.values, s=5, label='Data')
#axs[0,1].plot(range_x, fpl(range_x, exp_in) * count_nodes, 'k-', lw=1, alpha=.75, label='Power Law fit')
axs[0,1].set_yscale('log')
axs[0,1].set_xscale('log')
axs[0,1].set_title('Multiplicity of In-degrees for Talk pages [log-log]')
axs[0,1].set_xlim(1, max_degree)
axs[0,1].set_ylim(1, max_multiplicity)
axs[0,1].legend()

axs[1, 0].scatter(out_degrees_counts.index, out_degrees_counts.values, color="red", s=5, label='Data')
# hist, bin_edges, _ = axs[1, 0].hist(talk_page_in_degrees.values(), bins=no_bins_sturges, color="red", edgecolor='white', label='Data')
# axs[1, 0].set_xticks(bin_edges)
# axs[1, 0].set_xticklabels(['%.0f' % val for val in bin_edges], rotation=45)
axs[1,0].set_title('Multiplicity of Out-degrees for Users')
axs[1,0].legend()

axs[1,1].scatter(out_degrees_counts.index, out_degrees_counts.values, s=5, label='Data', color="red")
#axs[1,1].plot(range_x, fpl(range_x, exp_out) * count_nodes, 'k-', lw=1, alpha=.75, label='Power Law fit')
axs[1,1].set_yscale('log')
axs[1,1].set_xscale('log')
axs[1,1].set_title('Multiplicity of Out-degrees for Users [log-log]')
axs[1,1].set_xlim(1, max_degree)
axs[1,1].set_ylim(1, max_multiplicity)
axs[1,1].legend()

for ax in axs.flat:
    ax.set(xlabel='Degree', ylabel='Multiplicity')

fig.tight_layout()

# Sentiment analysis per user

For this, we will first need to extract all the comments from all pages, as well as the author of the comment
Then we will assign all comment texts to a single author, and run sentiment analysis on the texts.

In [68]:
wikipage_folder = pathlib.Path("./page_contents/")
filenames = list(wikipage_folder.rglob("*.txt"))

#worker_results = [parse_comments_from_pages(filenames[:100])]

with Pool(12) as pool:
    # perform calculations
    worker_results = pool.map(utils.parse_comments_from_pages, utils.chunk_list(filenames[:2000], 20))

failed to parse: page_contents/Talk:Economy.txt
failed to parse: page_contents/Talk:City.txt
failed to parse: page_contents/Talk:Sport of athletics.txt
failed to parse: page_contents/Talk:Scientific Revolution.txt
failed to parse: page_contents/Talk:Weak interaction.txt
failed to parse: page_contents/Talk:Crustacean.txt
failed to parse: page_contents/Talk:The Buddha.txt
failed to parse: page_contents/Talk:Natural rubber.txt
failed to parse: page_contents/Talk:News.txt
failed to parse: page_contents/Talk:Japan/Archive 9.txt
failed to parse: page_contents/Talk:Immanuel Kant/Archive 6.txt
failed to parse: page_contents/Talk:Johann Sebastian Bach/Archive 15.txt
failed to parse: page_contents/Talk:Argentina/Archive 7.txt
failed to parse: page_contents/Talk:United Kingdom/Archive 25.txt
failed to parse: page_contents/Talk:Mahatma Gandhi/Archive 9.txt
failed to parse: page_contents/Talk:Wolfgang Amadeus Mozart/Archive 2.txt
failed to parse: page_contents/Talk:Mahatma Gandhi/Archive 8.txt
fail

In [69]:
def append_or_create_list_in_dict(dict, key, value):
    if(key not in dict):
        dict[key] = []
    
    dict[key].append(value)

def append_comment_to_talkpage(page_dict, filepath, comment):
    '''Append a comment to a list of comments on that talk page. Will collect all comments for a specific talk
    page, including comments archived talk pages'''
    filepath_parts = filepath.split('/')

    if(len(filepath_parts) > 2):
        append_or_create_list_in_dict(page_dict, filepath_parts[1], comment)

    else:
        filename_parts = filepath_parts[1].split('.')
        append_or_create_list_in_dict(page_dict,filename_parts[0], comment)

In [70]:
author_dict = {}
list_for_df = []
page_dict = {}

# iterate over the results by the workers
# and transform the output into a dictionary with the users as keys
# and their comments as text
for worker_result in worker_results:
    for filepath, page in worker_result:
        for subsection in page["sections"]:
            if subsection.get("heading"):
                for comments in subsection.get("comments"):
                    if comments.get("author"):
                        for author, comment in utils.parse_comment_subcomment(comments):
                            append_or_create_list_in_dict(author_dict, author, comment)
                            append_comment_to_talkpage(page_dict, filepath, comment)
                            list_for_df.append([author, comment, filepath])

## Analyze sentiment on per-author basis

In [71]:
import utils
# show the top 5 authors written the most text in comment pages
# before tokenizing the comments
items = author_dict.items()
items_sorted = sorted(items, key=lambda x: len(x[1]), reverse=True)
[(author, len(utils.flatten(comments))) for author, comments in items_sorted][:5]

[('Rick Norwood', 449219),
 ('RJII', 385753),
 ('Peter jackson', 528358),
 ('RJHall', 309918),
 ('DeCausa', 287810)]

In [72]:
labMT = pd.read_csv("./labMT.txt", sep="\t")
# to facilitate happiness_average value lookup
labMT.set_index("word", inplace=True)

# Do sentiment analysis
# code taken from assignment 2

def sentiment(tokens):
    if(len(tokens) == 0):
        return
    freq = FreqDist(tokens)

    # filter for the vocabulary we can evaluate with LabMT
    vocab = list(filter(lambda word: word in labMT.index, np.unique(tokens)))

    # array of each token's average happiness weighted by the token's frequency
    weighted_happiness = np.fromiter((freq[word] * labMT.loc[word].happiness_average for word in vocab), dtype=float)
    # each token's frequency
    word_frequencies = np.fromiter((freq[word] for word in vocab), dtype=float)
    return np.sum(weighted_happiness) / np.sum(word_frequencies)

In [73]:
sentiments = {}
for author, text in author_dict.items():
    text = [utils.tokenize_custom(s) for s in text]
    text = utils.flatten(text)
    # compute sentiment for individual rapper wiki page
    if len(text) > 100:
        sentiment_value = sentiment(text)
        if(sentiment_value):
            sentiments[author] = sentiment_value

sentiments_df = pd.DataFrame({"Author": sentiments.keys(), "comment_happiness": sentiments.values()})
# use the author's name as index
sentiments_df.set_index("Author", inplace=True)
sentiments_df.head()

,comment_happiness
Author,
Dolphin51,5.364909
Rracecarr,5.376242
Spinningspark,5.381130
MrAureliusR,5.277685
DVdm,5.445121


In [74]:
print("Top 10 authors with the happiest comments:")
best_sentiments = sentiments_df.sort_values("comment_happiness", ascending=False).head(10)
best_sentiments

Top 10 authors with the happiest comments:


,comment_happiness
Author,
Lia 199712,5.897957
2A02:CE0:1801:50E:C51D:9330:A910:98D3,5.879540
Bkusmono,5.855294
76.2.40.115,5.841782
Balboadancer,5.769604
80.216.66.6,5.765306
Godar75,5.758889
Lwarnecke,5.758868
Praxaquilani,5.757143


In [75]:
print("Top 10 authors with the saddest comments:")
worst_sentiments = sentiments_df.sort_values("comment_happiness", ascending=True).head(10)
worst_sentiments

Top 10 authors with the saddest comments:


,comment_happiness
Author,
93.229.148.172,4.734500
Yuvalkatz,4.878079
71.211.175.77,4.886935
Bertus,4.903033
Hoxbar,4.904769
Dallas84,4.921635
Guard Chasseur,4.930728
108.170.154.240,4.951125
LongShaunSilvr,4.981053


In [76]:
# this shows the original comment before tokenizin
[s for s in author_dict.get(best_sentiments.iloc[0].name)]

[' == Semi-protected edit request on 29 May 2020 ==\n \n {{edit semi-protected|Great Barrier Reef|answered=yes}}\n One of Australia’s most remarkable natural gifts, the Great Barrier Reef is blessed with the breathtaking beauty of the world’s largest coral reef. The reef contains an abundance of marine life and comprises of over 3000 individual reef systems and coral cays and literally hundreds of picturesque tropical islands with some of the worlds most beautiful sun-soaked, golden beaches.\n Because of its natural beauty, the Great Barrier Reef has become one of the worlds most sought after tourist destinations.A visitor to the Great Barrier Reef can enjoy many experiences including snorkelling, scuba diving, aircraft or helicopter tours, bare boats (self-sail), glass-bottomed boat viewing, semi-submersibles and educational trips, cruise ship tours, whale watching and swimming with dolphins [[User:Lia 199712|Lia 199712]] ([[User talk:Lia 199712|talk]]) 02:25, 29 May 2020 (UTC)\n']

In [77]:
[s for s in author_dict.get(worst_sentiments.iloc[0].name)]

[' == Copper Vectoring ==\n \n Im Artikel steht der Satz \'\'\'"Kupfer leitet den elektrischen Strom sehr gut (58 · 106 S/m)."\'\'\' \n <br>Das Kupfernetz der Deutschen Telekom in Deutschland besteht zu 100 Prozent aus Kupfer. Die elektrische Spannung V beträgt zwischen 60-70 Volt. Der elektrische Stromfluss A ist variabel (Elektronik bis 5 Ampere). Die Bundesnetzagentur hat entschieden, dass im Netz der Deutschen Telekom der \'\'\'Bitstrom\'\'\' (Datenpakete) mit 100 MBit/s (300 MBit/s - Leitung/virtuelle Leitung/Leitung-) fließen soll. Dazu investiert die Deutsche Telekom ab 2013 zirka 200 Euro pro Haushalt, bei 24 Millionen Haushalten sind das  4,8 Milliarden Euro. Dabei wird die Kabelverteilertechnik in 330.000 Kabelverteilern in Deutschland durch Technik der Firma [[Alcatel Lucent]] ersetzt. Auf Seiten der Telekomkunden und Mitbewerberkunden werden entsprechende 100 MBit/s-Modems benötigt.  [[Special:Contributions/93.229.148.172|93.229.148.172]] ([[User talk:93.229.148.172|talk]])

### Results from per-author sentiment analysis

What we have seen is that comment sentiment mostly correlates with the topic they are discussing. E.g.: topics discussing the pages fear, Nazi, Hitler have a very low score, but comments discussing e.g. happiness, or the great barrier reef have very high sentiments.

These results show, that it is not beneficial to use the labMT approach to find toxic comments. Different methods should be used for our problem.
To show this also concretely, we will show a statistical correlation between talk page sentiment & article page sentiment.

## Per-talkpage sentiment analysis

In [78]:
sentiments_pages = {}
for pagename, text in page_dict.items():
    text = [utils.tokenize_custom(s) for s in text]
    text = utils.flatten(text)
    # compute sentiment for individual rapper wiki page
    if len(text) > 100:
        sentiment_value = sentiment(text)
        if(sentiment_value):
            sentiments_pages[pagename] = sentiment_value

sentiments_pages_df = pd.DataFrame({"page": sentiments_pages.keys(), "comment_happiness": sentiments_pages.values()})
# use the author's name as index
sentiments_pages_df.set_index("page", inplace=True)
sentiments_pages_df.head()

,comment_happiness
page,
Talk:Force,5.344659
Talk:Mediterranean Sea,5.391800
Talk:Orbit,5.323326
Talk:Dante Alighieri,5.413513
Talk:Pablo Picasso,5.554139


In [79]:
print("Top 10 happiest talk pages:")
best_sentiments = sentiments_pages_df.sort_values("comment_happiness", ascending=False).head(10)
best_sentiments

Top 10 happiest talk pages:


,comment_happiness
page,
Talk:Book,5.656556
Talk:Play (activity),5.640506
Talk:Solar energy,5.629508
Talk:Computer science,5.616852
Talk:Ecology,5.611111
Talk:Dream,5.606200
Talk:Memory,5.577665
Talk:Colombia,5.564483
Talk:Intelligence,5.562511


In [80]:
print("Top 10 saddest talk pages:")
worst_sentiments = sentiments_pages_df.sort_values("comment_happiness", ascending=True).head(10)
worst_sentiments

Top 10 saddest talk pages:


,comment_happiness
page,
Talk:Common cold,5.118947
Talk:Mental disorder,5.122759
Talk:Hospital,5.158393
Talk:Terrorism,5.168383
Talk:Nervous system,5.170948
Talk:Cold War,5.171320
Talk:Allergy,5.173241
Talk:War,5.185000
Talk:Fear,5.188344


### Extract article page sentiment



### Per-page sentiment analysis results



## Toxic comment extraction & analysis

In [81]:
df_comments = pd.DataFrame(list_for_df, columns = ['Author', 'Comment', 'Filename']) 

In [82]:
# imports for loading pickles
from nltk.tokenize import word_tokenize
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk

In [83]:
# Flatten the dictionary values
#flattened_data = [(key, value) for key, values in author_dict.items() for value in values]

# Create a DataFrame from the flattened data
#df = pd.DataFrame(flattened_data, columns=['Author', 'Comment'])

# load the vectorizer
with open('./sentiment-models/vectorizer.pkl', 'rb') as file:
    vec = pickle.load(file)

In [84]:
# create sparse TF-IDF matrix with vectorizer trained on kaggle toxic comment dataset

comments_sparse = vec.transform(df_comments['Comment'])

In [ ]:
# Classify comments according to the following categories:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

preds = np.zeros((len(df_comments), len(label_cols)))

for i, j in enumerate(label_cols):
    print('predict', j)
    with open(f'./sentiment-models/{j}.pkl', 'rb') as file:
        m,r = pickle.load(file)
    preds[:,i] = m.predict_proba(comments_sparse.multiply(r))[:,1]
    df_comments[j] = preds[:,i]

# this created a matrix where each new column for label_cols contains the probability that a comment is in
# the category with the same title as the column name 

predict toxic
predict severe_toxic
predict obscene
predict threat
predict insult
predict identity_hate


In [ ]:
# output all offensive comments
authors = []
for row in df_comments[df_comments["toxic"] > 0.9].iterrows():
    if row[1]["Author"]:
        authors.append(row[1]["Author"])
    print(row[1]["Filename"], row[1]["Author"], row[1]["toxic"], row[1]["Comment"])

page_contents/Talk:Hormone.txt 5.103.42.242 0.9200454171210526  == Are you dumb ==
 
 Idk [[Special:Contributions/5.103.42.242|5.103.42.242]] ([[User talk:5.103.42.242|talk]]) 18:29, 16 April 2023 (UTC)
page_contents/Talk:Scramble for Africa.txt DePiep 0.9962537910110114  :::::Sure you don't have to reply. But unless you ''quote me saying what you put in my mouth'', you can just as well shut up and fuck off. -[[User:DePiep|DePiep]] ([[User talk:DePiep|talk]]) 11:08, 2 December 2014 (UTC)

page_contents/Talk:Russia.txt GreenMeansGo 0.9062575750097962  :::Yeah, well I can call Joe Biden a big dumb idiot, and say he's too old to be using Legos, and that his feet probably smell bad, and somehow, I have no fear I'll be poisoned or haphazardly fall out a window. [[User:GreenMeansGo|<span style="font-family:Impact"><span style="color:#07CB4B">G</span><span style="color:#449351">M</span><span style="color:#35683d">G</span></span>]][[User talk:GreenMeansGo#top|<sup style="color:#000;font-family

In [ ]:
seen = set()
multiple_offenders = [x for x in authors if x in seen or seen.add(x)]    

# number of people that have written multiple toxic comments
len(set(multiple_offenders))


0

In [ ]:
# output all toxic comments of multiple offenders
for row in df_comments[df_comments["toxic"] > 0.9].iterrows():
    if row[1]["Author"] in multiple_offenders:
        print(f'{row[1]["Filename"]}, Author: {row[1]["Author"]}, Comment: {" ".join(word_tokenize(row[1]["Comment"]))}')